In [13]:
import splitfolders
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Split dataset into train (70%), val (20%), test (10%) with stratified sampling
original_dataset_dir = "./dataset"
output_dir = "./datasplit"

splitfolders.ratio(
    original_dataset_dir,
    output=output_dir,
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    move=False,
)

# Define paths
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Image dimensions and parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode="nearest",
)

# Validation and test generators
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

# Validation generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)

# Test generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)



Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Found 74 images belonging to 4 classes.


In [17]:

# Load MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [18]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model with frozen layers
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)




Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2723 - loss: 1.9752 - val_accuracy: 0.3431 - val_loss: 1.4458 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.3222 - loss: 1.6372 - val_accuracy: 0.4015 - val_loss: 1.3206 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.3639 - loss: 1.3813 - val_accuracy: 0.4234 - val_loss: 1.2537 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.3997 - loss: 1.3267 - val_accuracy: 0.4599 - val_loss: 1.1937 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.4140 - loss: 1.2526 - val_accuracy: 0.4818 - val_loss: 1.1334 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4579 - loss: 1.1988 - val_accuracy: 0.4453 - val_loss: 1.1350 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4679 - loss:

In [20]:
# Fine-tune the base model
base_model.trainable = True

# Freeze earlier layers for gradual unfreezing
for layer in base_model.layers[:80]:
    layer.trainable = False

# Re-compile for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [21]:
# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)




Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.3964 - loss: 1.3803 - val_accuracy: 0.5182 - val_loss: 1.0512 - learning_rate: 1.0000e-05
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.4412 - loss: 1.2106 - val_accuracy: 0.4891 - val_loss: 1.0885 - learning_rate: 1.0000e-05
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.4238 - loss: 1.1784 - val_accuracy: 0.4818 - val_loss: 1.1271 - learning_rate: 1.0000e-05
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5091 - loss: 1.0871 - val_accuracy: 0.4818 - val_loss: 1.1576 - learning_rate: 1.0000e-05
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5087 - loss: 1.0744 - val_accuracy: 0.4818 - val_loss: 1.1587 - learning_rate: 2.0000e-06


In [22]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 411ms/step - accuracy: 0.4609 - loss: 1.1484
Test Accuracy: 0.50
